# lstead 'class project' - organize
- renumbers course notebooks
    - makes a tedious task easy
- uses 'Permutator' module
- html "user interface"
    - uses flask to 'serve' interface
    - uses 'jinja2' template engine
- 320 lines
- a little tricky

In [5]:
import random

class Permutator:
    "Takes a list and permutes it in various ways"

    def __init__(self, lst):
        "Takes a list to permute. Input list will not be modified"
        self.original = lst
        self.reset()
        
    def __repr__(self):
        return str(self.lst)
    
    def __iter__(self):
        return iter(self.lst)
    
    def reset(self):
        "use the original list"
        self.lst = self.original[:]
          
    def swap(self, selects):
        "if two rows are selected, they will be swapped"
        if len(selects) == 2:
            lft, rht = selects
            self.lst[lft], self.lst[rht] = self.lst[rht], self.lst[lft]

    def permute(self, selects):
        "selected rows will be permuted"
        vals = [self.lst[s] for s in selects]
        random.shuffle(vals)
        for s, v in zip(selects, vals):
            self.lst[s] = v
            
    def up(self, selects):
        "moves selected rows up with wraparound"
        selects = sorted(selects)
        if selects[0]  == 0:
            row = self.lst.pop(0)
            self.lst.append(row)
        else:
            for s in selects:
                self.swap([s-1, s])
                
    def down(self, selects):
        "moves selected rows down with wraparound"
        selects = sorted(selects, reverse=True)
        last = len(self.lst)-1
        if selects[0] == last:
            row = self.lst.pop()
            self.lst.insert(0, row)
        else:
            for s in selects:
                self.swap([s, s + 1])

    def right(self, selects):
        "alias for down"
        self.down(selects)
    
    def left(self, selects):
        "alias for up"
        self.up(selects)


In [7]:
#!/Users/lstead/opt/anaconda3/bin/python 
 
# Flask server example

# hit flask server with curl    
# curl -H foo:bar -H another:header -H and:another   -i http://127.0.0.1:7777/

from flask import Flask, Response, request
import os
import re
#from Permutator import *
from jinja2 import Environment

app = Flask('Flask')

In [8]:
pre='''
<!DOCTYPE html>
<html>
<style>
input.red {
  color: rgb(255,0,0);
}
</style>
<body>

<form action="/answer" method="get">
  <input type="submit" value="All" name="op">
  <input type="submit" value="Clear" name="op">
  <input type="submit" value="Swap" name="op">
  <input type="submit" value="Permute" name="op">
  <input type="submit" value="Up" name="op">
  <input type="submit" value="Down" name="op">
  <input type="submit" value="Renumber" name="op">
  <input class="red" type="submit" value="Rename" name="op">
  <input class="red" type="submit" value="Reset" name="op">


<table >

  <tr>
    <th>Select</th>
    <th>File</th> 
  </tr>

'''

In [9]:
post='''
</table>


</form>
  
<hr>

help

</body>
</html>
'''

HTML='''
<!DOCTYPE html>
<html>
<style>
input.red {
  color: rgb(255,0,0);
}
</style>
<body>

<form action="/answer" method="get">
  <input type="submit" value="All" name="op">
  <input type="submit" value="Clear" name="op">
  <input type="submit" value="Swap" name="op">
  <input type="submit" value="Permute" name="op">
  <input type="submit" value="Up" name="op">
  <input type="submit" value="Down" name="op">
  <input type="submit" value="Renumber" name="op">
  <input class="red" type="submit" value="Rename" name="op">
  <input class="red" type="submit" value="Reset" name="op">


<table >

  <tr>
    <th>Select</th>
    <th>File</th> 
  </tr>

{{ rows }}

</table>
</form>
<hr>

help

</body>
</html>
'''

In [10]:
class frow:
    '''Holds file row info'''
    def __init__(self, name, num):
        self.name = name
        self.original = name
        self.num = num
        # Will this row be permutted?
        self.on = False

    def __repr__(self):
        return f'frow({self.name}, {self.num}, {self.on})'
    
    def row(self):
        "Generate HTML for this row"
        s = '''<tr>
    <td><input type="checkbox" name="{}" value="{}" {}></td>
    <td>{}</td>
  </tr>'''
        html =  s.format('on',# if self.on else 'off',
                         self.name, 
                         'checked' if self.on else '' ,
                         self.name)
        return html
        
    def rename(self):
        "Rename the file on disk"
        if self.original != self.name:
            print(f'rename: {self.original} to {self.name}')
            os.rename(self.original, self.name)

In [11]:
perm = []

def prows():
    "Print rows"
    print()
    for row in perm:
        print(row)

def makepage():
    s = ''
    for row in perm:
        s += '\n' + row.row()

    return pre + s + post

def makepage():
    "Make HTML for all rows."
    rows = ''
    for row in perm:
        rows += '\n' + row.row()
    
    out = Environment().from_string(HTML).render(rows=rows)

    return out


@app.route('/form')
def mkform():
    "Make HTML form to click on."
    global perm
    files = [file for file in os.listdir('.') \
            if re.match('week[0-9]-[0-9][0-9]-.*', file)]
    files.sort()
    perm = Permutator([frow(file, j) for j, file in enumerate(files)])
    return makepage()

@app.route('/answer')
def answer():
    "Process result of a click."
    args = request.args
    print('args', request.args)
    ons = request.args.getlist('on')
    # [j for j in range(len(rows)) if str(j) in request.args]
    print('ons', ons)
    selects = []
    for j, row in enumerate(perm):
        if row.name in ons:
            print('row ', row.name)
            selects.append(j)
            row.on = True
        else:
            row.on = False
    
    op = request.args['op']
    print('op is:', op)
    
    if op == 'Clear':
        for row in perm:
            row.on = False
    
    if op == 'All':
        for row in perm:
            row.on = True
    
    if op == 'Swap':
        perm.swap(selects)
    
    if op == 'Permute':
        perm.permute(selects)
    
    if op == 'Up':
        perm.up(selects)
    
    if op == 'Down':
        perm.down(selects)
    
    if op == 'Renumber':
        for j, row in enumerate(perm):
            row.oldname = row.name
            row.name = re.sub('-[0-9][0-9]-', '-{:02}-'.format(j+1), row.name)
    
    if op == 'Reset':
        perm.reset()
        for row in perm:
            row.on = False
    
    if op == 'Rename':
        for row in perm:
            row.rename()
        return mkform()
    return makepage()